# Python Maps - Leaflet Example

for more learning examples using leaflet visit [jupyter-widgets/ipyleaflet Notebook Examples](https://github.com/jupyter-widgets/ipyleaflet/tree/master/examples)

## Setup

In [1]:
%%capture
!pip install ipyleaflet
!pip install ipywidgets

In [2]:
import os
import json
import random
import requests
import pandas
from ipywidgets import *
from ipyleaflet import *
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
# CONSTANTS
MAP_LAT=40.3900
MAP_LON=-86.8220
MAP_CENTER = (MAP_LAT, MAP_LON)

In [4]:
# FUNCTIONS
def print_basemaps():
    basemap_list = basemaps.keys()
    print("\n","Basemaps")
    print("=========")
    print("-", "\n- ".join(basemap_list))

def print_basemap_themes(name):
    basemap_theme_list = basemaps[name].keys()
    print("\n", name, "Themes")
    print("=========")
    print("\n".join(basemap_theme_list))

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice([
            'red', 
            'yellow', 
            'green', 
            'orange'
        ]),
    }

def fetch_json(url):
    r = requests.get(tipp_cbg_url)
    text = r.content.decode("utf-8")
    data = json.loads(text)
    return data

def unique(list1):
    list_set = set(list1)
    unique_list = (list(list_set))
    return unique_list

In [5]:
# PREVIEW LEAFLET MAP BASELAYERS
# ------------------------------
# see: https://leaflet-extras.github.io/leaflet-providers/preview/
# see: https://ipyleaflet.readthedocs.io/en/latest/api_reference/tile_layer.html
print_basemaps()
print_basemap_themes("Esri")


 Basemaps
- OpenStreetMap
- OpenTopoMap
- Hydda
- Esri
- HikeBike
- MtbMap
- CartoDB
- NASAGIBS
- Strava
- Stamen

 Esri Themes
WorldStreetMap
DeLorme
WorldTopoMap
WorldImagery
NatGeoWorldMap


## Mapping Census Block Groups and Tippecanoe County

In [7]:
tipp_cbg_url = 'https://raw.githubusercontent.com/ironhacks/Tutorials-COVID-19/master/resources/tippecanoe-county-census-blocks.json'

data = fetch_json(tipp_cbg_url)

m = Map(
    center=MAP_CENTER, 
    zoom=10,
    basemap=basemaps.Stamen.Toner,
)

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 
        'dashArray': '9', 
        'fillOpacity': 0.4,
        'weight': 1
    },
    hover_style={
        'color': 'white', 
        'dashArray': '0', 
        'fillOpacity': 0.3
    },
    style_callback=random_color
)

m.add_layer(geo_json)

display(m)

Map(center=[40.39, -86.822], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

## Plotting Points of Interest from Google BigQuery

In [8]:
BIGQUERY_PROJECT = 'ironhacks-covid19-data'
BIGQUERY_KEYPATH = '/home/jovyan/key.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = BIGQUERY_KEYPATH
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

WEEKLY_PATTERNS_QUERY = """
SELECT 
  poi_id, 
  poi_cbg,
  location_name,
  latitude,
  longitude,
  top_category,
  raw_visit_counts
FROM `ironhacks_covid19_competition.weekly_patterns` 
WHERE week_number = 11
"""

query_job = bigquery_client.query(WEEKLY_PATTERNS_QUERY)
poi_geo = query_job.to_dataframe()

In [9]:
poi_geo

,poi_id,poi_cbg,location_name,latitude,longitude,top_category,raw_visit_counts
0,6a2d40f8-c477-49b6-a5d5-69c31f001afc,181570016001,Auto Express,40.367561,-86.868565,None,23
1,49636347-5a9d-4bf3-9d9a-c21a4ba7d8f1,181570003001,Del Real Auto Connection,40.438210,-86.859945,None,9
2,d98b37cd-ed64-456c-b580-c2c299c9f6d6,181570012001,Bear's Auto Sales,40.409919,-86.869882,None,11
3,efb4f5e7-28c2-47be-9d47-254280b8458e,181570001002,Gta Auto Sales,40.440608,-86.877960,None,2
4,16f4e696-2065-4854-8ba6-a8cab2b67bf7,181570001002,Steve Firestone Auto Sales,40.439060,-86.880571,None,22
...,...,...,...,...,...,...,...
1607,50c0e1b7-fca7-42c7-9a03-ae365b04a85f,181570052001,West Lafayette Fire Department Station 2,40.451514,-86.915309,"Justice, Public Order, and Safety Activities",9
1608,79d4b9e1-f93f-4fb4-aa5a-092e976d4e33,181570104001,Purdue University Fire Department,40.404468,-86.894375,"Justice, Public Order, and Safety Activities",32
1609,4fd58993-41ea-4db9-9a23-fa044395d052,181570018002,Lafayette Fire Department Station 5,40.424580,-86.839675,"Justice, Public Order, and Safety Activities",4
1610,ddfdb9d2-0f5a-46d8-8996-c437332d96a5,181570007001,Lafayette Fire Department Station 3,40.417589,-86.877738,"Justice, Public Order, and Safety Activities",29


## Marker Cluster with Popups

In [11]:
m = Map(center=MAP_CENTER, zoom=10)

markers = []

for index, point in poi_geo.iterrows():
    marker = Marker(
        location=[point.latitude, point.longitude],
        draggable=False,
    )
    marker_message = HTML()
    marker_message.value = "<strong>" + str(point.location_name) + "</strong>" + "<br>Visit Count: " + str(point.raw_visit_counts)
    marker.popup = marker_message
    markers.append(marker)

m.add_layer(MarkerCluster(markers=markers))

display(m)

Map(center=[40.39, -86.822], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [10]:
circle_map = Map(
    basemap=basemaps.CartoDB.Positron,
    center=MAP_CENTER, 
    zoom=14,
)

VISIT_COUNT_MIN = poi_geo["raw_visit_counts"].min()
VISIT_COUNT_MAX = poi_geo["raw_visit_counts"].max()
CIRCLE_SCALE = 150 # meters

def get_circle_radius(value):
    return int((value / VISIT_COUNT_MAX) * CIRCLE_SCALE)

circles = []

for index, point in poi_geo.iterrows():
    circle = Circle()
    circle.location=[point.latitude, point.longitude]
    circle.radius=get_circle_radius(point.raw_visit_counts)
    circle.color="red"
    circle.stroke=True
    circle.opacity=.8
    circle.fill=True
    circle.fill_color="red"
    circle.fill_opacity=.5
    circle.weight=2

    circle_message = HTML()
    circle_message.value = "<strong>" + str(point.location_name) + "</strong>" + "<br>Visit Count: " + str(point.raw_visit_counts)
    circle.popup = circle_message
    
    circles.append(circle)

circle_group = LayerGroup(layers=circles)
circle_map.add_layer(circle_group)

display(circle_map)

Map(center=[40.39, -86.822], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

## Advanced Mapping Techniques

### Multiple Tile Layers with Interaction

In [12]:
# INTERACTIVE MAP

basemap_example_map = Map(
    center=MAP_CENTER, 
    zoom=5
)

ee_basemaps={}

# Handles Dropdown control event
def on_click(change):
    basemap_name = change['new']
    old_basemap = basemap_example_map.layers[-1]
    basemap_example_map.substitute_layer(old_basemap, ee_basemaps[basemap_name])

# Loops through all ipyleaflet basemaps
for item in basemaps.values():
    try:
        name = item['name']
        basemap = 'basemaps.{}'.format(name)
        ee_basemaps[name] = basemap_to_tiles(eval(basemap))
    except:
        for sub_item in item:
            name = item[sub_item]['name']
            basemap = 'basemaps.{}'.format(name)
            basemap = basemap.replace('Mids', 'Modis')
            ee_basemaps[name] = basemap_to_tiles(eval(basemap))

# Adds a Dropdown widget
dropdown = Dropdown(
    options=list(ee_basemaps.keys()),
    value='OpenStreetMap.Mapnik',
    description='Basemaps'
)

dropdown.observe(on_click, 'value')

# Adds control to the map
basemap_control = WidgetControl(widget=dropdown, position='topright')
basemap_example_map.add_control(basemap_control)

display(basemap_example_map)

Map(center=[40.39, -86.822], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

### Multiple Maps

In [14]:
map2 = Map(center=[43.6, 1.44], zoom=10)
HBox([m, circle_map])

---

For more examples for mapping with leaflet in python visit the [official docs](https://ipyleaflet.readthedocs.io/en/latest/index.html)